In [1]:
import os
import tarfile
import urllib
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# <b>California Housing dataset</b>

- <span style = 'font-size:1.5em;line-height:1.5em'>캘리포니아 지역에서 발견된 주택과 1990년 인구조사 데이터를 기반으로 한 몇 가지 요약 통계에 관한 데이터</span>



In [2]:
df = pd.read_csv('data/housing/housing.csv')

# <b>Column Definition and Key value</b>


| Variable           | Definition                          | Key                                                     |
|--------------------|-------------------------------------|---------------------------------------------------------|
| longitude          | 경도                                |                                                         | 
| latitude           | 위도                                |                                                         |
| housing_median_age | 블록 내 주택들의 중간 연식          |                                                         |
| total_rooms        | 블록 내 주택들의 총 방의 수         |                                                         |
| total_bedrooms     | 블록 내 주택들의 총 침실 수         |                                                         |
| population	     | 블록 내 거주하는 총 주민 수         |                                                         |
| households	     | 블록 내 거주하는 총 가구 수         |                                                         |
| median_income	     | 블록 내 거주하는 가구들의 중위 소득 |                                                         |
| median_house_value | 블록 내 존재하는 주택들의 중위 가격 |                                                         |
| ocean_proximity	 | 바닷가를 기준으로 한 주택의 위치    |'NEAR BAY', '<1H OCEAN', 'INLAND', 'NEAR OCEAN', 'ISLAND'|

## <b>실습파일 "data exploration with sklearn.ipynb"로 부터 알 수 있는 사실</b>
- <span style = 'font-size:1.3em;line-height:1.5em'>주택 가격(<code>median_house_value</code>)는 중위 소득(<code>median_income</code>)과 양의 상관 관계</span>
- <span style = 'font-size:1.3em;line-height:1.5em'>일직선으로 보이는 값(500000, 450000, 350000, 280000) 근처의 값들을 제거해주는게 모델링 시 더 좋을 수도 있음</span>

# <b>2. 데이터 전처리</b>

## <b>(0) 비정상 데이터 제거</b>

- <span style = 'font-size:1.5em;line-height:1.5em'>비정상적인 `median_house_value >= 500000`, `housing_median_age>=50`인 부분을 제거</span>

In [3]:
df = df[(df['housing_median_age'] < 50) & 
        (df['median_house_value'] < 500000)].reset_index(drop=True)
#어느 하나라도 해당하는 데이터를 날려서 새로운 데이터를 만듦-> 다음과 같은 조건을 만족시키는 애들만 가져갈거야

## <b>(1) train-test split</b>
- <span style = 'font-size:1.5em;line-height:1.8em'>ML모델을 구축할 때, 학습과 평가를 하기 위한 dataset을 반드시 나누어야 합니다.</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>이에 대한 자세한 내용은 추후에 알아보도록 합시다.</span>


In [4]:
from sklearn.model_selection import train_test_split


In [ ]:
'''train_test_split(
    *arrays,
    test_size=None,
    train_size=None,
    random_state=None,
    shuffle=True, #순서대로 할건지 셔플링할건지 
    stratify=None,
)'''

In [9]:
# df_trn: 학습(train) 데이터
# df_tst: 시험(test) 데이터
df_trn, df_tst = train_test_split(df, test_size=0.2)
print(df_trn.shape, df_tst.shape, df.shape) #행,열

(14710, 10) (3678, 10) (18388, 10)


- <span style = 'font-size:1.5em;line-height:1.8em'>본격적인 전처리 전에 특성에 따라 column들을 다음과 같이 분리하겠습니다.</span>

In [10]:
all_cols = [x for x in df.columns]

geo_cols = ['longitude', 'latitude'] # 지리적 특성 column
y_cols = ['median_house_value'] # 종속 변수() column
cat_cols = ['ocean_proximity'] # 범주형 독립 변수(X) column
num_cols = [x for x in all_cols if x not in geo_cols + y_cols + cat_cols] # 수치형 독립 변수(X) column

In [11]:
print(all_cols)
print(geo_cols)
print(y_cols)
print(cat_cols)
print(num_cols)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']
['longitude', 'latitude']
['median_house_value']
['ocean_proximity']
['housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']


## <b>(2) 결측치 처리하기</b>
- <span style = 'font-size:1.5em;line-height:1.5em'>거의 모든 data에는 결측치가 존재합니다. ML 모델을 학습하기 위해서는 이를 처리해야 합니다.</span>
    - <span style = 'font-size:1.3em;line-height:1.5em'><b>Method 1: </b>Remove missing data - 결측치가 존재하는 행 제거하기</span>
    - <span style = 'font-size:1.3em;line-height:1.5em'><b>Method 2: </b>Impute missing data - 결측치를 특정 값으로 채워넣기</span>
<br></br>
    
- <span style = 'font-size:1.5em;line-height:1.5em'>이 data에는 `total_bedrooms`만 결측치가 존재합니다. 이를 어떻게 처리하는지 봅시다.</span>


In [12]:
df_trn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14710 entries, 16055 to 13046
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           14710 non-null  float64
 1   latitude            14710 non-null  float64
 2   housing_median_age  14710 non-null  float64
 3   total_rooms         14710 non-null  float64
 4   total_bedrooms      14563 non-null  float64
 5   population          14710 non-null  float64
 6   households          14710 non-null  float64
 7   median_income       14710 non-null  float64
 8   median_house_value  14710 non-null  float64
 9   ocean_proximity     14710 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.2+ MB


In [13]:
df_tst.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3678 entries, 7257 to 15448
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           3678 non-null   float64
 1   latitude            3678 non-null   float64
 2   housing_median_age  3678 non-null   float64
 3   total_rooms         3678 non-null   float64
 4   total_bedrooms      3634 non-null   float64
 5   population          3678 non-null   float64
 6   households          3678 non-null   float64
 7   median_income       3678 non-null   float64
 8   median_house_value  3678 non-null   float64
 9   ocean_proximity     3678 non-null   object 
dtypes: float64(9), object(1)
memory usage: 316.1+ KB


### <b>Method 1. Remove missing data</b>

In [14]:
df_trn_removed = df_trn[df_trn['total_bedrooms'].isnull() == False].reset_index(drop=True)
df_tst_removed = df_tst[df_tst['total_bedrooms'].isnull() == False].reset_index(drop=True)

# 결측치 제거 전 데이터 수
print(f"결측치 제거 전: train data 수={df_trn.shape[0]}, test data 수={df_tst.shape[0]}")
# 결측치 제거 후 데이터 수
print(f"결측치 제거 후: train data 수={df_trn_removed.shape[0]}, test data 수={df_tst_removed.shape[0]}")

결측치 제거 전: train data 수=14710, test data 수=3678
결측치 제거 후: train data 수=14563, test data 수=3634


### <b>Method 2. Impute Missing data</b>

- <span style = 'font-size:1.5em;line-height:1.8em'>데이터를 제거하는 대신 중간값(median)으로 채워 넣겠습니다.</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>그런데, 범주형(categorical) 데이터에서는 중간값이라는 개념이 없습니다.</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>수치형(numerical), 범주형 데이터로 나누고, 수치형 데이터만 채워넣도록 하겠습니다. </span>

In [15]:
all_cols = df_trn.columns

df_trn_num = df_trn[num_cols] # 학습 데이터 중, 수치형 변수만
df_trn_cat = df_trn[cat_cols] # 학습 데이터 중, 범주형 변수만
df_trn_geo = df_trn[geo_cols] # 학습 데이터 중, 지리적 특성 관련 변수만
df_trn_y = df_trn[y_cols] # 학습 데이터 중, 종속 변수만

df_tst_num = df_tst[num_cols] # 시험 데이터 중, 수치형 변수만
df_tst_cat = df_tst[cat_cols] # 시험 데이터 중, 범주형 변수만
df_tst_geo = df_tst[geo_cols] # 시험 데이터 중, 지리적 특성 관련 변수만
df_tst_y = df_tst[y_cols] # 시험 데이터 중, 종속 변수만

In [16]:
df_trn_num

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
16055,28.0,3081.0,460.0,1260.0,461.0,7.5372
11315,16.0,2987.0,864.0,1240.0,755.0,2.8231
7970,34.0,81.0,26.0,53.0,14.0,1.4091
2607,35.0,2053.0,412.0,1193.0,387.0,2.7500
12238,32.0,2086.0,458.0,1355.0,412.0,2.5238
...,...,...,...,...,...,...
12226,36.0,3337.0,687.0,2388.0,589.0,2.9628
12671,38.0,2065.0,374.0,812.0,343.0,3.1250
1052,16.0,2176.0,464.0,1410.0,434.0,3.5436
12346,24.0,6573.0,1235.0,2904.0,1202.0,3.0651


- <span style = 'font-size:1.5em;line-height:1.5em'>train data에서 각 column별로 어떤 값으로 결측치를 채워넣을지 결정</span>

In [17]:
from sklearn.impute import SimpleImputer

In [14]:
imputer = SimpleImputer(strategy='median')
imputer.fit(df_trn_num)
print(imputer.statistics_) # 각 column마다 이 값으로 결측치가 채워짐. 3번째에 해당하는 값이 total_bedrooms column의 빈칸 채우는 값

[  27.     2155.      444.     1210.      417.        3.4704]


- <span style = 'font-size:1.5em;line-height:1.5em'>train data를 impute한 결과 산출</span>

In [15]:
# df_trn_num의 결측치를 impute한 결과(단, numpy array 형태)
X_trn_num = imputer.transform(df_trn_num)

In [16]:
X_trn_num

array([[5.0000e+00, 3.0960e+03, 5.4500e+02, 1.7600e+03, 5.1900e+02,
        4.5701e+00],
       [1.4000e+01, 1.6921e+04, 2.8370e+03, 2.5240e+03, 1.2620e+03,
        7.6281e+00],
       [2.1000e+01, 3.1460e+03, 5.9500e+02, 1.5800e+03, 5.1300e+02,
        2.7857e+00],
       ...,
       [8.0000e+00, 2.0270e+03, 4.8000e+02, 1.7810e+03, 4.4700e+02,
        3.0806e+00],
       [3.2000e+01, 2.2380e+03, 4.7100e+02, 1.2920e+03, 4.6700e+02,
        1.1705e+00],
       [3.7000e+01, 7.3600e+02, 1.4800e+02, 3.3900e+02, 1.4000e+02,
        2.2875e+00]])

- <span style = 'font-size:1.5em;line-height:1.8m'><code>set_output(transform='pandas')</code></span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>impute된 결과물이 numpy array입니다.</span>
        - <span style = 'font-size:1.1em;line-height:1.8em'>각 column들이 어떤 녀석인지 알수가 없군요...</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>impute된 결과물이 pandas DataFrame이면 알기 쉽겠네요!</span>
        - <span style = 'font-size:1.1em;line-height:1.8em'><code>set_output(transform='pandas')</code>을 사용해서 pandas DataFrame형태의 output을 만들어봅시다!</span>

In [17]:
imputer = SimpleImputer(strategy='median').set_output(transform='pandas')
imputer.fit(df_trn_num)
print(imputer.statistics_) # 각 column마다 이 값으로 결측치가 채워짐. 5번째 값이 total_bedrooms column의 빈칸 채우는 값

[  27.     2155.      444.     1210.      417.        3.4704]


- <span style = 'font-size:1.5em;line-height:1.8em'><b>[주의]</b> test data에서 채워넣는 값과 train data에서 채워넣는 값은 반드시 동일해야 합니다!!</span>
- <span style = 'font-size:1.5em;line-height:1.8em'>test data의 비어있는 값도 train data에서 impute한 값으로 채워 넣기</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>test data에서는 별도로 <code>fit()</code>을 적용하지 않습니다!!</span>

In [18]:
X_trn_num = imputer.transform(df_trn_num)
X_trn_num.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
17216,5.0,3096.0,545.0,1760.0,519.0,4.5701
11045,14.0,16921.0,2837.0,2524.0,1262.0,7.6281
17661,21.0,3146.0,595.0,1580.0,513.0,2.7857
13058,16.0,2173.0,399.0,1460.0,393.0,4.2614
8745,45.0,350.0,444.0,225.0,72.0,1.8942


In [19]:
X_tst_num = imputer.transform(df_tst_num)
X_tst_num.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
16370,18.0,2677.0,785.0,1391.0,656.0,2.5067
14581,39.0,2614.0,634.0,1862.0,654.0,1.9238
15253,17.0,1826.0,544.0,1325.0,532.0,1.2762
15215,26.0,3563.0,579.0,1479.0,575.0,5.9522
1617,11.0,1013.0,188.0,410.0,158.0,4.8250


In [20]:
#뮤시해도됨

# # df_trn_num의 column과 index를 활용하여 numpy array 형태의 결과물을 pandas DataFrame으로 변환
# df_trn_num = pd.DataFrame(X, columns=df_trn_num.columns, index=df_trn_num.index)

# # Numerical과 Categorical 부분을 결합
# df_trn_imputed = pd.concat((df_trn_num, df_trn_cat), axis=1)

# # Column의 순서를 원래대로 변환
# df_trn_imputed = df_trn_imputed[all_cols]

- <span style = 'font-size:1.5em;line-height:1.8em'>수치형 변수들만 있는 데이터의 imputation 결과를 확인해봅시다</span>

In [21]:
df_trn_num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14710 entries, 17216 to 919
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   housing_median_age  14710 non-null  float64
 1   total_rooms         14710 non-null  float64
 2   total_bedrooms      14558 non-null  float64
 3   population          14710 non-null  float64
 4   households          14710 non-null  float64
 5   median_income       14710 non-null  float64
dtypes: float64(6)
memory usage: 804.5 KB


In [22]:
X_trn_num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14710 entries, 17216 to 919
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   housing_median_age  14710 non-null  float64
 1   total_rooms         14710 non-null  float64
 2   total_bedrooms      14710 non-null  float64
 3   population          14710 non-null  float64
 4   households          14710 non-null  float64
 5   median_income       14710 non-null  float64
dtypes: float64(6)
memory usage: 804.5 KB


In [23]:
df_tst_num.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3678 entries, 16370 to 3762
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   housing_median_age  3678 non-null   float64
 1   total_rooms         3678 non-null   float64
 2   total_bedrooms      3639 non-null   float64
 3   population          3678 non-null   float64
 4   households          3678 non-null   float64
 5   median_income       3678 non-null   float64
dtypes: float64(6)
memory usage: 201.1 KB


In [24]:
X_tst_num.info() #2번 데이터 값이 채워짐

<class 'pandas.core.frame.DataFrame'>
Index: 3678 entries, 16370 to 3762
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   housing_median_age  3678 non-null   float64
 1   total_rooms         3678 non-null   float64
 2   total_bedrooms      3678 non-null   float64
 3   population          3678 non-null   float64
 4   households          3678 non-null   float64
 5   median_income       3678 non-null   float64
dtypes: float64(6)
memory usage: 201.1 KB


## <b>(3) 새로운 변수 생성하기</b>

- <span style = 'font-size:1.5em;line-height:1.8em'>여러 논의를 수행한 끝에 다음과 같은 결과를 얻었다고 가정하자.</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>주택 가격은 전체 방 개수, 침실 개수 보다 <b>(침실 개수) / (전체 방 개수)</b>에 더 많은 영향이 있다.</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>이 비율을 모델링에 활용하기 위해, <b>새로운 변수를 생성</b>해야 한다!</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>그 대신, <b>침실 개수와 전체 방 개수는 모델링 과정에서 제외</b>한다.</span>

In [25]:
X_trn_num.head()

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income
17216,5.0,3096.0,545.0,1760.0,519.0,4.5701
11045,14.0,16921.0,2837.0,2524.0,1262.0,7.6281
17661,21.0,3146.0,595.0,1580.0,513.0,2.7857
13058,16.0,2173.0,399.0,1460.0,393.0,4.2614
8745,45.0,350.0,444.0,225.0,72.0,1.8942


- <span style = 'font-size:1.5em;line-height:1.8em'>위의 "2. 결측치 처리(imputation)"가 완료된 데이터를 활용하여 <code>bedrooms_per_room</code> 변수 생성</span>

In [26]:
X_trn_num['bedrooms_per_room'] = X_trn_num['total_bedrooms'] / X_trn_num['total_rooms']
X_tst_num['bedrooms_per_room'] = X_tst_num['total_bedrooms'] / X_tst_num['total_rooms']

- <span style = 'font-size:1.5em;line-height:1.8em'>과연 <code>bedrooms_per_room</code>이 <code>total_bedrooms</code>, <code>total_rooms</code>보다 <code>median_house_value</code>에 영향이 있을까???</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>`median_house_value`에 대해서 `total_rooms`, `total_bedrooms`보다 `bedrooms_per_room`의 상관관계가 더 크다.(절대값 기준)</span>

In [27]:
X_tmp = pd.concat((X_trn_num, df_trn_y), axis=1)

In [28]:
corr_matrix = X_tmp.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

median_house_value    1.000000
median_income         0.666395
total_rooms           0.156328
households            0.101720
total_bedrooms        0.078867
population            0.026234
housing_median_age    0.016209
bedrooms_per_room    -0.224193
Name: median_house_value, dtype: float64

- <span style = 'font-size:1.5em;line-height:1.8em'>오케이- 그럼 모델 생성시 <code>bedrooms_per_room</code>을 활용하자</span>
- <span style = 'font-size:1.5em;line-height:1.8em'>대신, <code>bedrooms_per_room</code>를 산출하는데 사용한 <code>total_bedrooms</code>, <code>total_rooms</code>는 제거하자!</span>

In [29]:
X_trn_num = X_trn_num.drop(['total_bedrooms', 'total_rooms'], axis=1)
X_tst_num = X_tst_num.drop(['total_bedrooms', 'total_rooms'], axis=1)

In [30]:
X_trn_num.head()

,housing_median_age,population,households,median_income,bedrooms_per_room
17216,5.0,1760.0,519.0,4.5701,0.176034
11045,14.0,2524.0,1262.0,7.6281,0.167661
17661,21.0,1580.0,513.0,2.7857,0.189129
13058,16.0,1460.0,393.0,4.2614,0.183617
8745,45.0,225.0,72.0,1.8942,1.268571


In [31]:
X_tst_num.head()

,housing_median_age,population,households,median_income,bedrooms_per_room
16370,18.0,1391.0,656.0,2.5067,0.293239
14581,39.0,1862.0,654.0,1.9238,0.242540
15253,17.0,1325.0,532.0,1.2762,0.297919
15215,26.0,1479.0,575.0,5.9522,0.162504
1617,11.0,410.0,158.0,4.8250,0.185587


## <b>(4) 수치형 변수 스케일링</b>

- <span style = 'font-size:1.5em;line-height:1.5em'>사용하는 ML모델에 따라서 데이터를 <b>표준 정규 분포 스케일로 표준화(Standardize)</b>하거나 <b>0-1스케일로 정규화</b> 해야할 때가 있습니다.</span>
    - <span style = 'font-size:1.3em;line-height:1.5em'>이러한 모델들은 변수간의 scale이 이질적이지 않고 정규 분포를 따른다는 가정하에 동작하기 때문에, 실제 데이터가 이러한 가정을 벗어난다면 잘 동작하지 않는 것입니다. (ex: Linear Regression) </span>
<br></br>
    - <span style = 'font-size:1.2em;line-height:1.5em'><b>표준화(Standardization)</b></span>
    $$Z = {(x-\mu)\over\sigma} $$
    - <span style = 'font-size:1.2em;line-height:1.5em'><b>정규화(Normalization): (MinMaxScaling)</b></span>
    $$Z = {(x-x_{min})\over(x_{max}-x_{min})} $$

- <span style = 'font-size:1.5em;line-height:1.5em'>scikit-learn에서는 train, test set에서 같은 스케일로 데이터를 변환하도록 하는 기능을 제공합니다. </span>

In [32]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

- <span style = 'font-size:1.5em;line-height:1.8em'>각 column별로 standardize(normalize)하는 방식 학습하기</span>

In [33]:
# scaler = StandardScaler().set_output(transform='pandas')
scaler = MinMaxScaler().set_output(transform='pandas')
scaler.fit(X_trn_num)

MinMaxScaler()

- <span style = 'font-size:1.5em;line-height:1.8em'>train data를 normalize한 결과 산출</span>

In [34]:
X_trn_num_scaled = scaler.transform(X_trn_num)

- <span style = 'font-size:1.5em;line-height:1.8em'><b>[주의]</b> test data에서 normalize하는 방식과 train data에서 normalize하는 방식은 반드시 동일해야 합니다!!</span>
- <span style = 'font-size:1.5em;line-height:1.8em'>train data를 normalize하는 방식으로 test data를 normalize하기</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>test data에서는 별도로 <code>fit()</code>을 적용하지 않습니다!!</span>

- <span style = 'font-size:1.5em;line-height:1.8em'>test data를 normalize한 결과 산출</span>

In [35]:
X_tst_num_scaled = scaler.transform(X_tst_num)

- <span style = 'font-size:1.5em;line-height:1.8em'>결과를 비교해보자</span>

In [36]:
X_trn_num.head()

,housing_median_age,population,households,median_income,bedrooms_per_room
17216,5.0,1760.0,519.0,4.5701,0.176034
11045,14.0,2524.0,1262.0,7.6281,0.167661
17661,21.0,1580.0,513.0,2.7857,0.189129
13058,16.0,1460.0,393.0,4.2614,0.183617
8745,45.0,225.0,72.0,1.8942,1.268571


In [37]:
X_trn_num_scaled.head()

,housing_median_age,population,households,median_income,bedrooms_per_room
17216,0.083333,0.107778,0.096527,0.280700,0.075570
11045,0.270833,0.154644,0.235250,0.491593,0.070989
17661,0.416667,0.096737,0.095407,0.157639,0.082735
13058,0.312500,0.089376,0.073002,0.259410,0.079720
8745,0.916667,0.013618,0.013069,0.096157,0.673361


## <b>(5) 범주형(Categorical) 변수 처리하기</b>

- <span style = 'font-size:1.5em;line-height:1.8em'>ML알고리즘에 따라 범주형 변수를 그대로 사용할 수 있는 경우도 있고, 그렇지 않은 경우도 존재</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>그대로 사용하지 못하는 경우, numerical vector형태로 형태 변환을 해줘야 한다. </span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>여러가지 방법이 존재하지만 여기서는 가장 간단한 방법인 onehot encoding방법에 대해서만 알아보자.</span>

![onehot_encoding](img/onehot_encoding.png)

-출처: https://towardsdatascience.com/building-a-one-hot-encoding-layer-with-tensorflow-f907d686bf39

In [38]:
from sklearn.preprocessing import OneHotEncoder

- <span style = 'font-size:1.5em;line-height:1.8em'>train data에서 <code>ocean_proximity</code> column을 onehot encoding하는 과정 생성</span>

In [39]:
encoder = OneHotEncoder()
encoder.fit(df_trn_cat)

OneHotEncoder()

- <span style = 'font-size:1.5em;line-height:1.8em'>train data에서 <code>ocean_proximity</code>column을 onehot encoding!!</span>

In [40]:
X_trn_cat = encoder.transform(df_trn_cat)
X_trn_cat

<14710x5 sparse matrix of type '<class 'numpy.float64'>'
	with 14710 stored elements in Compressed Sparse Row format>

- <span style = 'font-size:1.5em;line-height:1.8m'><code>OneHotEncoder(sparse_output=False)</code></span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>위 onehot encoding한 결과물은 CSR(compressed sparse row) 형식</span>
        - <span style = 'font-size:1.1em;line-height:1.8em'>sparse: 거의 대부분의 값이 0으로 채워져 있는 matrix</span>
        - <span style = 'font-size:1.1em;line-height:1.8em'>이를 효율적으로 표현하기 위한 방식</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>뭔지 모르겠으니까 우리가 익숙한 numpy.ndarray형식으로 값을 반환해라!!!</span>
        - <span style = 'font-size:1.1em;line-height:1.8em'><code>sparse_output=False</code></span>

- <span style = 'font-size:1.5em;line-height:1.8m'><code>set_output(transform='pandas')</code></span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>onehot encoding된 결과물이 numpy array입니다.</span>
        - <span style = 'font-size:1.1em;line-height:1.8em'>각 column들이 어떤 녀석인지 알수가 없군요...</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>onehot encoding 결과물이 pandas DataFrame이면 알기 쉽겠네요!</span>
        - <span style = 'font-size:1.1em;line-height:1.8em'><code>set_output(transform='pandas')</code>을 사용해서 pandas DataFrame형태의 output을 만들어봅시다!</span>

In [41]:
encoder = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
encoder.fit(df_trn_cat)

OneHotEncoder(sparse_output=False)

- <span style = 'font-size:1.5em;line-height:1.5em'>train data에서 <code>ocean_proximity</code>column의 범주 값은 onehot encoding!!</span>

In [42]:
X_trn_cat = encoder.transform(df_trn_cat)

In [43]:
X_trn_cat.head()

,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
17216,0.0,1.0,0.0,0.0,0.0
11045,0.0,1.0,0.0,0.0,0.0
17661,0.0,1.0,0.0,0.0,0.0
13058,1.0,0.0,0.0,0.0,0.0
8745,0.0,0.0,0.0,1.0,0.0


- <span style = 'font-size:1.5em;line-height:1.8em'><b>[주의]</b> test data의 onehot encoding 방식과 train data에서 onehot encoding 방식은 반드시 동일해야 합니다!!</span>
- <span style = 'font-size:1.5em;line-height:1.8em'>train data를 onehot encoding한 방식으로 test data도 onehot encoding하기</span>
    - <span style = 'font-size:1.3em;line-height:1.8em'>test data에서는 별도로 <code>fit()</code>을 적용하지 않습니다!!</span>

In [44]:
X_tst_cat = encoder.transform(df_tst_cat)

In [45]:
X_trn_cat.head()

,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
17216,0.0,1.0,0.0,0.0,0.0
11045,0.0,1.0,0.0,0.0,0.0
17661,0.0,1.0,0.0,0.0,0.0
13058,1.0,0.0,0.0,0.0,0.0
8745,0.0,0.0,0.0,1.0,0.0


- <span style = 'font-size:1.5em;line-height:1.5em'>결과를 비교해보자</span>

In [46]:
df_trn_cat.head()

,ocean_proximity
17216,INLAND
11045,INLAND
17661,INLAND
13058,<1H OCEAN
8745,NEAR BAY


In [47]:
X_trn_cat.head()

,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
17216,0.0,1.0,0.0,0.0,0.0
11045,0.0,1.0,0.0,0.0,0.0
17661,0.0,1.0,0.0,0.0,0.0
13058,1.0,0.0,0.0,0.0,0.0
8745,0.0,0.0,0.0,1.0,0.0


In [48]:
df_tst_cat.head()

,ocean_proximity
16370,NEAR OCEAN
14581,INLAND
15253,<1H OCEAN
15215,<1H OCEAN
1617,INLAND


In [49]:
X_tst_cat.head()

,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
16370,0.0,0.0,0.0,0.0,1.0
14581,0.0,1.0,0.0,0.0,0.0
15253,1.0,0.0,0.0,0.0,0.0
15215,1.0,0.0,0.0,0.0,0.0
1617,0.0,1.0,0.0,0.0,0.0


## <b>(6) 수치형 변수, 범주형 변수 전처리한 결과 통합하기</b>

- <span style = 'font-size:1.5em;line-height:1.5em'>수치형 변수 전처리 결과</span>

In [50]:
# train data
X_trn_num_scaled.head()

,housing_median_age,population,households,median_income,bedrooms_per_room
17216,0.083333,0.107778,0.096527,0.280700,0.075570
11045,0.270833,0.154644,0.235250,0.491593,0.070989
17661,0.416667,0.096737,0.095407,0.157639,0.082735
13058,0.312500,0.089376,0.073002,0.259410,0.079720
8745,0.916667,0.013618,0.013069,0.096157,0.673361


In [51]:
# test data
X_tst_num_scaled.head()

,housing_median_age,population,households,median_income,bedrooms_per_room
16370,0.354167,0.085143,0.122106,0.138398,0.139700
14581,0.791667,0.114035,0.121733,0.098199,0.111960
15253,0.333333,0.081094,0.098954,0.053537,0.142261
15215,0.520833,0.090541,0.106983,0.376016,0.068167
1617,0.208333,0.024966,0.029126,0.298279,0.080798


- <span style = 'font-size:1.5em;line-height:1.5em'>범주형 변수 전처리 결과</span>

In [52]:
# train data
X_trn_cat.head()

,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
17216,0.0,1.0,0.0,0.0,0.0
11045,0.0,1.0,0.0,0.0,0.0
17661,0.0,1.0,0.0,0.0,0.0
13058,1.0,0.0,0.0,0.0,0.0
8745,0.0,0.0,0.0,1.0,0.0


In [53]:
# test data
X_tst_cat.head()

,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
16370,0.0,0.0,0.0,0.0,1.0
14581,0.0,1.0,0.0,0.0,0.0
15253,1.0,0.0,0.0,0.0,0.0
15215,1.0,0.0,0.0,0.0,0.0
1617,0.0,1.0,0.0,0.0,0.0


- <span style = 'font-size:1.5em;line-height:1.5em'>수치형 변수를 전처리한 결과와 범주형 변수를 전처리한 결과를 통합</span>

In [54]:
X_trn = pd.concat((df_trn_geo, df_trn_y, X_trn_num_scaled, X_trn_cat), axis=1)
X_tst = pd.concat((df_tst_geo, df_tst_y, X_tst_num_scaled, X_tst_cat), axis=1)

In [55]:
X_trn.head()

,longitude,latitude,median_house_value,housing_median_age,population,households,median_income,bedrooms_per_room,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
17216,-121.05,37.65,146400.0,0.083333,0.107778,0.096527,0.280700,0.075570,0.0,1.0,0.0,0.0,0.0
11045,-116.42,33.76,341700.0,0.270833,0.154644,0.235250,0.491593,0.070989,0.0,1.0,0.0,0.0,0.0
17661,-119.09,36.35,92700.0,0.416667,0.096737,0.095407,0.157639,0.082735,0.0,1.0,0.0,0.0,0.0
13058,-117.13,32.92,169600.0,0.312500,0.089376,0.073002,0.259410,0.079720,1.0,0.0,0.0,0.0,0.0
8745,-122.23,38.17,216700.0,0.916667,0.013618,0.013069,0.096157,0.673361,0.0,0.0,0.0,1.0,0.0


In [56]:
X_tst.head()

,longitude,latitude,median_house_value,housing_median_age,population,households,median_income,bedrooms_per_room,ocean_proximity_<1H OCEAN,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
16370,-122.03,36.96,232600.0,0.354167,0.085143,0.122106,0.138398,0.139700,0.0,0.0,0.0,0.0,1.0
14581,-121.27,38.13,70700.0,0.791667,0.114035,0.121733,0.098199,0.111960,0.0,1.0,0.0,0.0,0.0
15253,-119.69,34.42,253600.0,0.333333,0.081094,0.098954,0.053537,0.142261,1.0,0.0,0.0,0.0,0.0
15215,-119.75,34.50,438400.0,0.520833,0.090541,0.106983,0.376016,0.068167,1.0,0.0,0.0,0.0,0.0
1617,-120.84,38.77,184600.0,0.208333,0.024966,0.029126,0.298279,0.080798,0.0,1.0,0.0,0.0,0.0
